In [1]:
import numpy as np
import pandas as pd

init_notebook_mode(connected=True) 
np.set_printoptions(linewidth=160)

---

In [2]:
df_crimes = pd.read_csv("Crimes/table_4_crime_in_the_united_states_by_region_geographic_division_and_state_2014-2015_raw.csv")

In [2]:
df_ganja = pd.read_csv("marijuana-street-price-clean.csv")

---

In [4]:
df_crimes.head()

,Area,Year,Population2,Violent crime3,Unnamed: 4,Murder and nonnegligent manslaughter,Unnamed: 6,Rape (revised definition)4,Unnamed: 8,Rape (legacy definition)5,...,Property crime,Unnamed: 16,Burglary,Unnamed: 18,Larceny-theft,Unnamed: 20,Motor vehicle theft,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,NaN,NaN,NaN,,"Rate per \n100,000",,"Rate per \n100,000",,"Rate per \n100,000",,...,,"Rate per \n100,000",,"Rate per \n100,000",,"Rate per \n100,000",,"Rate per \n100,000",NaN,NaN
1,New England6,2014,14.689.812,42.550,"289,7",295,"2,0",4.559,"31,0",3.337,...,280.035,"1.906,3",53.116,"361,6",208.755,"1.421,1",18.164,"123,7",NaN,
2,NaN,2015,14.727.584,42.089,"285,8",321,"2,2",4.544,"30,9",3.464,...,256.342,"1.740,6",46.009,"312,4",192.654,"1.308,1",17.679,"120,0",NaN,NaN
3,NaN,Percent\nchange,,"-1,1","-1,3","+8,8","+8,5","-0,3","-0,6","+3,8",...,"-8,5","-8,7","-13,4","-13,6","-7,7","-7,9","-2,7","-2,9",NaN,NaN
4,Connecticut,2014,3.594.762,8.575,"238,5",89,"2,5",794,"22,1",579,...,69.326,"1.928,5",12.017,"334,3",51.195,"1.424,2",6.114,"170,1",NaN,NaN


In [10]:
df_ganja.head()


,State,HighQ,HighQN,MedQ,MedQN,LowQ,LowQN,date
0,Alabama,339.06,1042,198.64,933,149.49,123,2014-01-01
1,Alaska,288.75,252,260.60,297,388.58,26,2014-01-01
2,Arizona,303.31,1941,209.35,1625,189.45,222,2014-01-01
3,Arkansas,361.85,576,185.62,544,125.87,112,2014-01-01
4,California,248.78,12096,193.56,12812,192.92,778,2014-01-01


In [6]:
# CRIME DATASET PREPROCESSING
# Remove unused columns
del df_crimes['Unnamed: 23']
del df_crimes['Unnamed: 24']

In [7]:
# Remove percentage increase rows and unused rows
df_crimes = df_crimes.loc[df_crimes.index.map(lambda x : x%3 != 0 and x < 184)]

In [8]:
# Rename columns
_transl_table = str.maketrans({c:None for c in '0123456789,'})
_strip_junk = lambda x : x.translate(_transl_table).strip().replace('\n',' ').replace('  ',' ')
df_crimes.columns = np.array(list(map(_strip_junk, df_crimes.columns.values )))

l = []
for i,val in enumerate(df_crimes.columns):
    if val == 'Unnamed:':
        l.append(val_prev + ', rate per 100,000')
    else:
        l.append(val)
        val_prev = val
df_crimes.columns = l
del l,i,val,val_prev

In [9]:
# Rename states, replicate names where missing
df_crimes.Area[0::2] = df_crimes.Area[0::2].map(_strip_junk)
df_crimes.Area[1::2] = df_crimes.Area[0::2]

In [10]:
df_crimes.sort_values(by=['Area', 'Year'], inplace=True)

In [11]:
df_crimes.index = range(1, 1+len(df_crimes.index))

In [12]:
df_crimes.head()

,Area,Year,Population,Violent crime,"Violent crime, rate per 100,000",Murder and nonnegligent manslaughter,"Murder and nonnegligent manslaughter, rate per 100,000",Rape (revised definition),"Rape (revised definition), rate per 100,000",Rape (legacy definition),...,Aggravated assault,"Aggravated assault, rate per 100,000",Property crime,"Property crime, rate per 100,000",Burglary,"Burglary, rate per 100,000",Larceny-theft,"Larceny-theft, rate per 100,000",Motor vehicle theft,"Motor vehicle theft, rate per 100,000"
1,Alabama,2014,4.846.411,20.727,"427,7",276,"5,7",2.005,"41,4",1.425,...,13.744,"283,6",154.087,"3.179,4",39.723,"819,6",104.223,"2.150,5",10.141,"209,2"
2,Alabama,2015,4.858.979,22.952,"472,4",348,"7,2",2.039,"42,0",1.456,...,15.954,"328,3",144.746,"2.978,9",35.255,"725,6",99.156,"2.040,7",10.335,"212,7"
3,Alaska,2014,737.046,4.684,"635,5",41,"5,6",771,"104,6",553,...,3.243,"440,0",20.334,"2.758,9",3.150,"427,4",15.445,"2.095,5",1.739,"235,9"
4,Alaska,2015,738.432,5.392,"730,2",59,"8,0",901,"122,0",650,...,3.671,"497,1",20.806,"2.817,6",3.511,"475,5",15.249,"2.065,1",2.046,"277,1"
5,Arizona,2014,6.728.783,26.422,"392,7",311,"4,6",3.272,"48,6",2.356,...,16.614,"246,9",213.406,"3.171,5",43.412,"645,2",152.683,"2.269,1",17.311,"257,3"


In [13]:
s1 = set(df_crimes['Area'][::2])
s2 = set(df_ganja['State'])

/print s1-s2
print("="*120)
/print s2-s1

{'New England', 'South Atlantic', 'East North Central', 'Mountain', 'Middle Atlantic', 'West South Central', 'East South Central', 'West North Central', 'Puerto Rico', 'Pacific'}
set()


In [14]:
df_crimes.to_csv("Crimes/table_4_crime_in_the_united_states_by_region_geographic_division_and_state_2014-2015.csv")

In [15]:
df_ganja.columns.values

array(['State', 'HighQ', 'HighQN', 'MedQ', 'MedQN', 'LowQ', 'LowQN', 'date'], dtype=object)

In [16]:
df_ganja['Amount'] = df_ganja.HighQ * df_ganja.HighQN + df_ganja.MedQ * df_ganja.MedQN

In [17]:
df_joined = df_crimes[['Area', 'Population', 'Violent crime']][df_crimes.Year == '2014'].merge(
    df_ganja[['State', 'Amount']].groupby('State').sum(),
    left_on = 'Area', right_on = 'State'
#     pd.DataFrame.merge()
)

In [18]:
df_joined.head()

,Area,Population,Violent crime,Amount
0,Alabama,4.846.411,20.727,3.271121e+08
1,Alaska,737.046,4.684,9.021175e+07
2,Arizona,6.728.783,26.422,5.236423e+08
3,Arkansas,2.966.835,14.248,1.796538e+08
4,California,38.792.291,153.763,3.084078e+09


<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>